In [3]:
pip install mediapipe opencv-python pandas numpy


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import mediapipe as mp
import pandas as pd
import time


start_time=time.time()


# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Video file path (CHANGE THIS TO YOUR VIDEO FILE PATH)
video_path = "/Users/anand/Desktop/gesturio.ai/ML/asl_demo_video.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize storage
landmark_data = []
frame_num = 0  # Frame counter

# Load MediaPipe Holistic
with mp_holistic.Holistic(static_image_mode=False, 
                          min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit when video ends

        frame_num += 1  # Track frame number
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        results = holistic.process(rgb_frame)  # Get holistic results

        # Function to save landmarks
        def save_landmarks(landmarks, type_name):
            if landmarks:
                for i, lm in enumerate(landmarks.landmark):
                    landmark_data.append({
                        "frame": frame_num,
                        "row_id": f"{frame_num}_{type_name}_{i}",
                        "type": type_name,
                        "landmark_index": i,
                        "x": lm.x,
                        "y": lm.y,
                        "z": lm.z
                    })

        # Extract landmarks
        save_landmarks(results.pose_landmarks, "pose")
        save_landmarks(results.face_landmarks, "face")
        save_landmarks(results.left_hand_landmarks, "left_hand")
        save_landmarks(results.right_hand_landmarks, "right_hand")

        # Optional: Display video with landmarks
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        cv2.imshow("Video Processing", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



# Save the data to a CSV file
df = pd.DataFrame(landmark_data)
df.to_csv("landmarks_output.csv", index=False)

print("✅ Landmark extraction completed! Data saved to 'landmarks_output.csv'.")
end_time=time.time()
execution_time=end_time-start_time
print(f"Execution Time: {execution_time:.6f} seconds")


I0000 00:00:1743190659.340612  161501 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
W0000 00:00:1743190659.414327  162384 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743190659.428214  162384 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743190659.429569  162386 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743190659.429755  162388 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743190659.430446  162391 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support 

✅ Landmark extraction completed! Data saved to 'landmarks_output.csv'.
Execution Time: 4.476397 seconds
